# IR_Assignment2_TiffanyHoeung

Question 3: Inverted Index with Counts

In [ ]:
from post_parser_record import PostParserRecord
post_reader = PostParserRecord("Posts_Coffee.xml")

In [ ]:
# imports
import nltk
from nltk.corpus import stopwords
from nltk.corpus.reader.tagged import word_tokenize
import re, string
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
word_dict = {}

# For each question
for answer_id in post_reader.map_questions:
  # Get the text
  text = (post_reader.map_questions[answer_id].title + " " + post_reader.map_questions[answer_id].body)

  # Remove punctuations, make lowercase
  token_words = re.sub("<.*?>|\\n|&quot;", " ", text.lower())
  token_words = word_tokenize(token_words.translate(str.maketrans('', '', string.punctuation)))

  # Go through all the words and add them to the dictionary with count
  for word in token_words:
    if word not in stopwords.words('english'):
      if word in word_dict:
        docs = word_dict[word]
        if answer_id not in docs:
          docs.update({answer_id:1})
        else:
          count = docs[answer_id]
          docs[answer_id] = count+1
        word_dict[word] = docs
      else:
        docs = {answer_id:1}
        word_dict[word] = docs

# For each question
for answer_id in post_reader.map_just_answers:
  # Get the text
  text = (post_reader.map_just_answers[answer_id].body)

  # Remove punctuations, make lowercase
  token_words = re.sub("<.*?>|\\n|&quot;", " ", text.lower())
  token_words = word_tokenize(token_words.translate(str.maketrans('', '', string.punctuation)))

  # Go through all the words and add them to the dictionary with count
  for word in token_words:
    if word not in stopwords.words('english'):
      if word in word_dict:
        docs = word_dict[word]
        if answer_id not in docs:
          docs.update({answer_id:1})
        else:
          count = docs[answer_id]
          docs[answer_id] = count+1
        word_dict[word] = docs
      else:
        docs = {answer_id:1}
        word_dict[word] = docs


In [ ]:
# Creating a TSV file with the inverted index
import csv, os
csvfilename = os.path.join("/content", 'CoffeTable.csv')

with open(csvfilename, 'w') as f:
  dict_writer = csv.DictWriter(f, ["Word", "Documents"], extrasaction='ignore', delimiter='\t')
  dict_writer.writeheader()
  for word in word_dict:
    dict_writer.writerow({"Word": word, "Documents": str(word_dict[word])})

Question 4: Generating TREC-formatted file

In [ ]:
from itertools import islice
# Sort our three queries : "espresso" "coffee" "turkish"
sorted_espresso = dict(sorted(word_dict["espresso"].items(), key=lambda item: item[1], reverse=True))
sorted_coffee = dict(sorted(word_dict["coffee"].items(), key=lambda item: item[1], reverse=True))
sorted_turkish = dict(sorted(word_dict["turkish"].items(), key=lambda item: item[1], reverse=True))

# combine coffee + turkish dictionaries; only take doc ids that exist in both dictionaries
sorted_turkishcoffee = {k: sorted_coffee.get(k, 0) + sorted_turkish.get(k, 0) for k in set(sorted_coffee) & set(sorted_turkish)}
sorted_turkishcoffee = dict(sorted(sorted_turkishcoffee.items(), key=lambda item: item[1], reverse=True))

# Print results
print("Espresso: " + str(sorted_espresso))
print("Turkish: " + str(sorted_turkish))
print("Coffee: " + str(sorted_coffee))
print("Turkish Coffee: " + str(sorted_turkishcoffee))


Espresso: {3269: 34, 1574: 22, 2095: 20, 283: 14, 5537: 14, 2077: 13, 2087: 12, 2116: 12, 3721: 12, 3438: 11, 1630: 10, 4895: 10, 3284: 9, 3956: 9, 288: 9, 1651: 9, 2208: 8, 5377: 8, 470: 8, 1840: 8, 3694: 8, 3764: 8, 4000: 8, 5837: 8, 2766: 7, 3252: 7, 4618: 7, 135: 7, 324: 7, 2102: 7, 2433: 7, 2687: 7, 3720: 7, 4259: 7, 4422: 7, 5057: 7, 5095: 7, 5685: 7, 5764: 7, 2791: 6, 3518: 6, 3768: 6, 3894: 6, 4279: 6, 4727: 6, 4880: 6, 5277: 6, 5730: 6, 5742: 6, 419: 6, 466: 6, 2316: 6, 2399: 6, 3184: 6, 3186: 6, 3771: 6, 3958: 6, 4041: 6, 4407: 6, 5528: 6, 5532: 6, 5536: 6, 5824: 6, 26: 5, 1860: 5, 2562: 5, 3362: 5, 3812: 5, 3953: 5, 4860: 5, 5152: 5, 5400: 5, 5786: 5, 5886: 5, 1526: 5, 2124: 5, 2644: 5, 2706: 5, 2710: 5, 2966: 5, 3402: 5, 3689: 5, 3927: 5, 4176: 5, 4262: 5, 4412: 5, 4683: 5, 4749: 5, 5544: 5, 5603: 5, 5608: 5, 5863: 5, 1572: 4, 1951: 4, 2017: 4, 2265: 4, 2350: 4, 2705: 4, 2745: 4, 2957: 4, 2959: 4, 3113: 4, 3434: 4, 4128: 4, 4175: 4, 4258: 4, 4261: 4, 4275: 4, 4401: 4, 4755:

In [ ]:
# Get all of the documents we want to add to the qrel file:
# Espresso: 3438, 2087, 2116, 3721, 2077, 283, 5537, 2095, 1574, 3269
# Turkish Coffee: 2392, 4407, 4273, 1833, 4185, 3101, 5095, 2647, 2379, 3939
espresso_documents = {3269: 0, 1574: 0, 2095: 1, 5537: 0, 283: 0, 2077: 0, 3721: 1,
                      2116: 1, 2087: 0, 3438: 0}

turkishcoffee_documents = {2392: 1, 4407: 0, 4273: 0, 1833: 0, 4185: 0, 3101: 0, 
                           5095: 1, 2647: 0, 2379: 1, 3939: 0}

In [ ]:
# qrel format: query_id 0 document_id relevance
# ex. : Q001 0 2208 1

qrel_filename = os.path.join("/content", 'CoffeePostsQrel.txt')

with open(qrel_filename, 'w') as f:
  for doc in espresso_documents:
    line = "Q001 0 " + str(doc) + " " + str(espresso_documents[doc])
    f.write(line + "\n")
  
  for doc in turkishcoffee_documents:
    line = "Q002 0 " + str(doc) + " " + str(turkishcoffee_documents[doc])
    f.write(line + "\n")

# qrel results format: query_id Q0 document_id rank score STANDARD
# ex. : Q001 Q0 2208 1 1 CoffeePost
qrelresults_filename = os.path.join("/content", 'CoffeePostsResults.txt')

with open(qrelresults_filename, 'w') as f:
  for doc in espresso_documents:
    line = "Q001 Q0 " + str(doc) + " 1 " + str(sorted_espresso[doc]) + " CoffeePosts"
    f.write(line + "\n")
  
  for doc in turkishcoffee_documents:
    line = "Q002 Q0 " + str(doc) + " 1 " + str(sorted_turkishcoffee[doc]) + " CoffeePosts"
    f.write(line + "\n")

Question 5: Calculating Kendall's Tau

In [ ]:
import scipy.stats as stats
p10 = [0.3566, 0.1818, 0.0584, 0.0519, 0.5479, 0.1883, 0.2974, 0.0039, 0.2403, 0.2390, 0.1390]
mmr = [0.6103, 0.4339, 0.1003, 0.0854, 0.8640, 0.2811, 0.4829, 0.0237, 0.4196, 0.3857, 0.2432]
tau, p_value = stats.kendalltau(p10, mmr)
print("Tau is " + str(tau) + " and p_value is " + str(p_value))

Tau is 0.8909090909090909 and p_value is 1.3728555395222063e-05
